# args & validate dataset

In [1]:
model_path = "E:\\data\\models\\gpt-boost\\gpt-tiny"
data_path = "E:\\data\\corpus\\gpt-boost\\novel.txt"
tokenizer_path = ".\\tokenizer"
tb_path = ".\\runs\\gpt-tiny-finetune"

In [2]:
n_embd=512
n_head=8
n_layer=4
n_positions=256
vocab_size=20000
finetuning_mode=True

In [3]:
learning_rate=1e-4
gradient_accumulation_steps=4
num_train_epochs=40
per_device_train_batch_size=16

# 1. create tokenizer

In [4]:
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained(tokenizer_path, max_len=256)

Special tokens have been added in the vocabulary, make sure the associated word emebedding are fine-tuned or trained.


# 2. initialize model

In [5]:
from transformers import GPT2LMHeadModel

if finetuning_mode:
    model = GPT2LMHeadModel.from_pretrained(model_path)
else:
    from transformers import GPT2Config
    config = GPT2Config(
        n_embd=n_embd,
        n_head=n_head,
        n_layer=n_layer,
        n_positions=n_positions,
        vocab_size=vocab_size
    )
    model = GPT2LMHeadModel(config=config)

In [6]:
model.num_parameters()
# => 22981632 parameters (约23M)

22981632

# 3. build training Dataset

In [7]:
from transformers import TextDataset

dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=data_path,
    block_size=256,
)

Like in the [`run_language_modeling.py`](https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_language_modeling.py) script, we need to define a data_collator.

This is just a small helper that will help us batch different samples of the dataset together into an object that PyTorch knows how to perform backprop on.

In [8]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

# 4. initialize our Trainer & Train & Save

In [9]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=model_path,
    overwrite_output_dir=True,
    no_cuda=False,
    do_train=True,
    fp16=True,
    logging_dir=tb_path,
    logging_steps=100,
    learning_rate=learning_rate,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    save_steps=1000,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

E:\Users\anshengqiang\anaconda3\lib\site-packages\transformers\trainer.py:245: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead.
  warnings.warn(


In [10]:
trainer.train()

{'loss': 3.6444317626953127, 'learning_rate': 9.991264849755416e-05, 'epoch': 0.034934497816593885, 'step': 100}
{'loss': 3.637578125, 'learning_rate': 9.982529699510832e-05, 'epoch': 0.06986899563318777, 'step': 200}
{'loss': 3.6367864990234375, 'learning_rate': 9.973794549266247e-05, 'epoch': 0.10480349344978165, 'step': 300}
{'loss': 3.63298095703125, 'learning_rate': 9.965059399021663e-05, 'epoch': 0.13973799126637554, 'step': 400}
{'loss': 3.634908447265625, 'learning_rate': 9.95632424877708e-05, 'epoch': 0.17467248908296942, 'step': 500}
{'loss': 3.627374267578125, 'learning_rate': 9.947589098532495e-05, 'epoch': 0.2096069868995633, 'step': 600}
{'loss': 3.633173828125, 'learning_rate': 9.938853948287911e-05, 'epoch': 0.2445414847161572, 'step': 700}
{'loss': 3.6273291015625, 'learning_rate': 9.930118798043327e-05, 'epoch': 0.2794759825327511, 'step': 800}
{'loss': 3.6208056640625, 'learning_rate': 9.921383647798742e-05, 'epoch': 0.314410480349345, 'step': 900}
{'loss': 3.6272094

E:\Users\anshengqiang\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{'loss': 3.6269921875, 'learning_rate': 9.903913347309575e-05, 'epoch': 0.38427947598253276, 'step': 1100}
{'loss': 3.6226806640625, 'learning_rate': 9.89517819706499e-05, 'epoch': 0.4192139737991266, 'step': 1200}
{'loss': 3.6258740234375, 'learning_rate': 9.886443046820406e-05, 'epoch': 0.45414847161572053, 'step': 1300}
{'loss': 3.62333984375, 'learning_rate': 9.877707896575821e-05, 'epoch': 0.4890829694323144, 'step': 1400}
{'loss': 3.61947265625, 'learning_rate': 9.868972746331237e-05, 'epoch': 0.5240174672489083, 'step': 1500}
{'loss': 3.6176318359375, 'learning_rate': 9.860237596086654e-05, 'epoch': 0.5589519650655022, 'step': 1600}
{'loss': 3.62626953125, 'learning_rate': 9.85150244584207e-05, 'epoch': 0.5938864628820961, 'step': 1700}
{'loss': 3.62984375, 'learning_rate': 9.842767295597485e-05, 'epoch': 0.62882096069869, 'step': 1800}
{'loss': 3.6247119140625, 'learning_rate': 9.834032145352901e-05, 'epoch': 0.6637554585152838, 'step': 1900}
{'loss': 3.63083984375, 'learning_r

{'loss': 3.634814453125, 'learning_rate': 9.74668064290706e-05, 'epoch': 1.0132751091703056, 'step': 2900}
{'loss': 3.602333984375, 'learning_rate': 9.737945492662475e-05, 'epoch': 1.0482096069868996, 'step': 3000}
{'loss': 3.60810546875, 'learning_rate': 9.72921034241789e-05, 'epoch': 1.0831441048034935, 'step': 3100}
{'loss': 3.6054296875, 'learning_rate': 9.720475192173306e-05, 'epoch': 1.1180786026200873, 'step': 3200}
{'loss': 3.61333984375, 'learning_rate': 9.711740041928722e-05, 'epoch': 1.1530131004366813, 'step': 3300}
{'loss': 3.607607421875, 'learning_rate': 9.703004891684137e-05, 'epoch': 1.1879475982532752, 'step': 3400}
{'loss': 3.607451171875, 'learning_rate': 9.694269741439553e-05, 'epoch': 1.222882096069869, 'step': 3500}
{'loss': 3.60982421875, 'learning_rate': 9.685534591194969e-05, 'epoch': 1.257816593886463, 'step': 3600}
{'loss': 3.61615234375, 'learning_rate': 9.676799440950385e-05, 'epoch': 1.2927510917030567, 'step': 3700}
{'loss': 3.61173828125, 'learning_rate

{'loss': 3.6258203125, 'learning_rate': 9.493361285814117e-05, 'epoch': 2.026550218340611, 'step': 5800}
{'loss': 3.6058984375, 'learning_rate': 9.484626135569533e-05, 'epoch': 2.061484716157205, 'step': 5900}
{'loss': 3.60408203125, 'learning_rate': 9.475890985324948e-05, 'epoch': 2.096419213973799, 'step': 6000}
{'loss': 3.593984375, 'learning_rate': 9.467155835080364e-05, 'epoch': 2.131353711790393, 'step': 6100}
{'loss': 3.59841796875, 'learning_rate': 9.458420684835779e-05, 'epoch': 2.166288209606987, 'step': 6200}
{'loss': 3.60515625, 'learning_rate': 9.449685534591195e-05, 'epoch': 2.2012227074235806, 'step': 6300}
{'loss': 3.597109375, 'learning_rate': 9.44095038434661e-05, 'epoch': 2.2361572052401746, 'step': 6400}
{'loss': 3.596328125, 'learning_rate': 9.432215234102026e-05, 'epoch': 2.2710917030567686, 'step': 6500}
{'loss': 3.60216796875, 'learning_rate': 9.423480083857443e-05, 'epoch': 2.3060262008733625, 'step': 6600}
{'loss': 3.6048828125, 'learning_rate': 9.414744933612

...

{'loss': 3.61671875, 'learning_rate': 7.999650593990217e-05, 'epoch': 8.001397379912664, 'step': 22900}
{'loss': 3.5809375, 'learning_rate': 7.990915443745632e-05, 'epoch': 8.036331877729257, 'step': 23000}
{'loss': 3.586015625, 'learning_rate': 7.982180293501048e-05, 'epoch': 8.071266375545852, 'step': 23100}
{'loss': 3.57203125, 'learning_rate': 7.973445143256464e-05, 'epoch': 8.106200873362445, 'step': 23200}
{'loss': 3.58140625, 'learning_rate': 7.96470999301188e-05, 'epoch': 8.14113537117904, 'step': 23300}
{'loss': 3.58625, 'learning_rate': 7.955974842767296e-05, 'epoch': 8.176069868995633, 'step': 23400}
{'loss': 3.57609375, 'learning_rate': 7.947239692522712e-05, 'epoch': 8.211004366812228, 'step': 23500}
{'loss': 3.591640625, 'learning_rate': 7.938504542278127e-05, 'epoch': 8.24593886462882, 'step': 23600}
{'loss': 3.584296875, 'learning_rate': 7.929769392033544e-05, 'epoch': 8.280873362445416, 'step': 23700}
{'loss': 3.5909375, 'learning_rate': 7.92103424178896e-05, 'epoch': 

{'loss': 3.61875, 'learning_rate': 7.746331236897276e-05, 'epoch': 9.014672489082969, 'step': 25800}
{'loss': 3.580859375, 'learning_rate': 7.737596086652691e-05, 'epoch': 9.049606986899564, 'step': 25900}
{'loss': 3.571328125, 'learning_rate': 7.728860936408107e-05, 'epoch': 9.084541484716157, 'step': 26000}
{'loss': 3.5884375, 'learning_rate': 7.720125786163522e-05, 'epoch': 9.119475982532752, 'step': 26100}
{'loss': 3.58421875, 'learning_rate': 7.711390635918938e-05, 'epoch': 9.154410480349345, 'step': 26200}
{'loss': 3.59015625, 'learning_rate': 7.702655485674354e-05, 'epoch': 9.18934497816594, 'step': 26300}
{'loss': 3.586953125, 'learning_rate': 7.693920335429769e-05, 'epoch': 9.224279475982533, 'step': 26400}
{'loss': 3.579140625, 'learning_rate': 7.685185185185185e-05, 'epoch': 9.259213973799127, 'step': 26500}
{'loss': 3.59484375, 'learning_rate': 7.6764500349406e-05, 'epoch': 9.29414847161572, 'step': 26600}
{'loss': 3.58109375, 'learning_rate': 7.667714884696017e-05, 'epoch'

{'loss': 3.59546875, 'learning_rate': 7.493011879804333e-05, 'epoch': 10.027947598253276, 'step': 28700}
{'loss': 3.58296875, 'learning_rate': 7.484276729559749e-05, 'epoch': 10.062882096069869, 'step': 28800}
{'loss': 3.5846875, 'learning_rate': 7.475541579315166e-05, 'epoch': 10.097816593886463, 'step': 28900}
{'loss': 3.58875, 'learning_rate': 7.466806429070581e-05, 'epoch': 10.132751091703057, 'step': 29000}
{'loss': 3.5846875, 'learning_rate': 7.458071278825997e-05, 'epoch': 10.167685589519651, 'step': 29100}
{'loss': 3.5790625, 'learning_rate': 7.449336128581412e-05, 'epoch': 10.202620087336244, 'step': 29200}
{'loss': 3.58421875, 'learning_rate': 7.440600978336828e-05, 'epoch': 10.237554585152838, 'step': 29300}
{'loss': 3.58203125, 'learning_rate': 7.431865828092244e-05, 'epoch': 10.272489082969432, 'step': 29400}
{'loss': 3.580625, 'learning_rate': 7.423130677847659e-05, 'epoch': 10.307423580786025, 'step': 29500}
{'loss': 3.585859375, 'learning_rate': 7.414395527603075e-05, '

{'loss': 3.613125, 'learning_rate': 7.248427672955975e-05, 'epoch': 11.006288209606987, 'step': 31500}
{'loss': 3.590078125, 'learning_rate': 7.239692522711391e-05, 'epoch': 11.04122270742358, 'step': 31600}
{'loss': 3.578203125, 'learning_rate': 7.230957372466806e-05, 'epoch': 11.076157205240175, 'step': 31700}
{'loss': 3.578203125, 'learning_rate': 7.222222222222222e-05, 'epoch': 11.111091703056768, 'step': 31800}
{'loss': 3.5775, 'learning_rate': 7.213487071977639e-05, 'epoch': 11.146026200873363, 'step': 31900}
{'loss': 3.578203125, 'learning_rate': 7.204751921733054e-05, 'epoch': 11.180960698689956, 'step': 32000}
{'loss': 3.581953125, 'learning_rate': 7.19601677148847e-05, 'epoch': 11.21589519650655, 'step': 32100}
{'loss': 3.582109375, 'learning_rate': 7.187281621243886e-05, 'epoch': 11.250829694323144, 'step': 32200}
{'loss': 3.577265625, 'learning_rate': 7.178546470999301e-05, 'epoch': 11.285764192139737, 'step': 32300}
{'loss': 3.5875, 'learning_rate': 7.169811320754717e-05, 

KeyboardInterrupt: 

### 🎉 Save final model (+ tokenizer + config) to disk

In [11]:
trainer.save_model(model_path)

#### ！！！注意，这里需要把词表拷贝到模型文件夹！！！

# 5. Check the model

In [14]:
from transformers import pipeline

generate = pipeline(
    "text-generation",
    model=model_path,
)

In [20]:
import time
time0 = time.time()
result = generate("""第八十四章 满月的呓语
　　克莱恩刚披上双排扣长礼服，拿起半高丝绸礼帽，往门口走去，忽然听见了层层回荡的虚幻祈求声。
　　谁？他微皱眉头，侧耳倾听了一下，但只能确认祈求者是一位女士，而且嗓音断断续续，似乎蕴藏着极大的痛苦。
　　想着也没什么特别紧要的事情，新晋“魔术师”克莱恩随手一扔，让半高丝绸礼帽又准确无误地挂到了衣帽架上，自身则返回卧室，逆走四步，进入巍峨雄伟的宫殿。
　　这一次，""", max_length=256, repetition_penalty=1, do_sample=True, top_k=40, temperature=1.2, num_return_sequences=3)
print(time.time()-time0)
for item in result:
  print(item["generated_text"])
  print()
  print("="*50)
  print()

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


1.5550296306610107
第八十四章 满月的呓语
　　克莱恩刚披上双排扣长礼服，拿起半高丝绸礼帽，往门口走去，忽然听见了层层回荡的虚幻祈求声。
　　谁？他微皱眉头，侧耳倾听了一下，但只能确认祈求者是一位女士，而且嗓音断断续续，似乎蕴藏着极大的痛苦。
　　想着也没什么特别紧要的事情，新晋“魔术师”克莱恩随手一扔，让半高丝绸礼帽又准确无误地挂到了衣帽架上，自身则返回卧室，逆走四步，进入巍峨雄伟的宫殿。
　　这一次，他借助“源堡”提供了力量，让“福根之犬”途径失控需要很久来提升力量的奥黛丽借助深红星辰来维持自己的身体，以维持自身的姿态，未去尝试祈求者，未变再出现较合适的仪式。
　　在灰雾之上，愚者祈祷者的灵性直觉是催眠的前置目标。
　　经过这次的事情，奥黛丽对“祈福人”途径有一定的了解，所以对晋升“愚者”有较有戒备了，因为有“愚者”力量带来的帮助，“魔术师”小姐能很快恢复正常，而且，这让她感觉得到


第八十四章 满月的呓语
　　克莱恩刚披上双排扣长礼服，拿起半高丝绸礼帽，往门口走去，忽然听见了层层回荡的虚幻祈求声。
　　谁？他微皱眉头，侧耳倾听了一下，但只能确认祈求者是一位女士，而且嗓音断断续续，似乎蕴藏着极大的痛苦。
　　想着也没什么特别紧要的事情，新晋“魔术师”克莱恩随手一扔，让半高丝绸礼帽又准确无误地挂到了衣帽架上，自身则返回卧室，逆走四步，进入巍峨雄伟的宫殿。
　　这一次，他没有尝试敲门，因为他能真切地感受到自身有“梦境占卜”的技巧。
　　这很说明，这是非常好的结果。
　　这是指它被灰雾包裹的状态，有特殊，有特殊的那种感觉，没有特殊的地方，克莱恩没有刻意去思考，只是根据预定，敲开门窗，看见里面一片漆黑，没有任何生气。
　　这……克莱恩暗自吸了口气道：
　　“你真是的，有的时候，我好像更喜欢这种场景。”
　　他旋即吸了口气，准备回房，准备起别的占卜。
　　可


第八十四章 满月的呓语
　　克莱恩刚披上双排扣长礼服，拿起半高丝绸礼帽，往门口走去，忽然听见了层层回荡的虚幻祈求声。
　　谁？他微皱眉头，侧耳倾听了一下，但只能确认祈求者是一位女士，而且嗓音断断续续，似乎蕴藏着极大的痛苦。
　　想着也没什么特别紧要的事情，新晋“魔术师”克莱恩随手一扔，让半高丝绸礼帽又准确无误地挂到了衣帽架上，自身则返回卧室，逆走四步，进入巍峨雄伟的宫殿。
　　这一次，他不再像